## 13.1 Ajax 的定义

Ajax 是 Asynchronous JavaScript and XML（异步 JavaScript 和 XML）的缩写（但不是首字母缩
写）

Ajax 本身并不是一种新技术，它是由几种长期存在的 Web 技术组合而成的：
- 使用 HTML 和 CSS 控制页面结构和表示方式；
- 使用 DOM 显示和操纵页面；
- 使用浏览器的 XMLHttpRequest 对象在客户机和服务器之间传输数据①
- 使用 XML 作为在客户机和服务器之间传输的数据的格式②
- 最后，使用 JavaScript 动态地显示所有内容并且提供交互功能。

## 13.2 读取服务器数据

使用 XMLHttpRequest 对象获得和显示来自服务器的数据。

In [ ]:
window.addEventListener("load",initAll,false);
var xhr = false;
// 它是一个 XMLHttpRequest 对象（或者说在初始化之后将成为 XMLHttpRequest 对象）。
// 目前，我们只需在任何函数之外创建它，使它成为全局变量。
function initAll() {
	document.getElementById("makeTextRequest").addEventListener("click",getNewFile,false);
	document.getElementById("makeXMLRequest").addEventListener("click",getNewFile,false);
}

function getNewFile(evt) {
	makeRequest(this.href);
	evt.preventDefault();
}

// P 274
// 这个对象是 window 的一个属性。
function makeRequest(url) {
	if (window.XMLHttpRequest) {
        // 创建一个新的 XMLHttpRequest 对象。
		xhr = new XMLHttpRequest();
	}
	else {
		if (window.ActiveXObject) {
			try {
				xhr = new ActiveXObject("Microsoft.XMLHTTP");
			}
			catch (e) {
			}
		}
	}

	if (xhr) {
        // 设置 xhr 的 readystatechange 事件处理程序。
        // 每当 xhr.readyState 属性值发生变化时，就会触发这个处理程序。
		xhr.addEventListener("readystatechange",showContents,false);

        // 调用 open()并且传递 3 个参数：一个 HTTP 请求方法（例如"GET"、"POST"或"HEAD"）
        //  URL 和一个布尔值，
        // 这个布尔值告诉服务器请求是否异步（也就是说，我们是否会等待请求完成）。
		xhr.open("GET", url, true);

        // 最后，我们用 send()发送刚才创建的请求。如果要请求 POST，就传递这里给出的参数。
		xhr.send(null);
	}
	else {
		document.getElementById("updateArea").innerHTML = "Sorry, but I couldn't create an XMLHttpRequest";
	}
}

// readyState 属性可能是几个值之一，而且每当服务器改变它的值时，就会触发 showContents()函数。
// 但是，在请求完成之前，我们实际上不希望执行任何操作（至少在这里不做什么）
function showContents() {
	if (xhr.readyState == 4) {
		if (xhr.status == 200) {
            // 。如果数据是 XML，responseXML 属性就包含数据。
			if (xhr.responseXML && xhr.responseXML.childNodes.length > 0) {
				var outMsg = getText(xhr.responseXML.getElementsByTagName("choices")[0]);
			}
			else {
                // 如果得到的数据不是有效的 XML，那么它就是文本文件。
				var outMsg = xhr.responseText;
			}
		}
		else {
			var outMsg = "There was a problem with the request " + xhr.status;
		}
        // 输出服务器返回的值
		document.getElementById("updateArea").innerHTML = outMsg;
	}
	
	function getText(inVal) {
		if (inVal.textContent) {
			return inVal.textContent;
		}
		return inVal.text;
	}
}

表13-2 readyState属性值
- 0 未初始化。对象不包含数据
- 1 正在加载。对象当前正在加载它的数据
- 2 已经加载。对象已经完成了数据加载
- 3 交互式的。即使对象还未完全加载完，用户也可能与对象进行交互
- 4 完成。对象已经完成了初始化

Ajax 调用的缺点之一是它们可能被缓存。也就是说，应用程序看似与服务器进行通信并且获得新数据，但是它实际上只是查看以前读取的数据。如果是这种情况，设置请求首部会有帮助。添加以下请求首部可以迫使服务器提供最新数据：
```javascript
    xhr.setRequestHeader("Cache-Control", "no-cache");
```

## 13.3 解析服务器数据


In [ ]:
window.addEventListener("load",initAll,false);
var xhr = false;

function initAll() {
	if (window.XMLHttpRequest) {
		xhr = new XMLHttpRequest();
	}
	else {
		if (window.ActiveXObject) {
			try {
				xhr = new ActiveXObject("Microsoft.XMLHTTP");
			}
			catch (e) {
			}
		}
	}

	if (xhr) {
		xhr.addEventListener("readystatechange",showPictures,false);
		xhr.open("GET", "flickrfeed.xml", true);
		xhr.send(null);
	}
	else {
		alert("Sorry, but I couldn't create an XMLHttpRequest");
	}
}

function showPictures() {
	var tempText = document.createElement("div");
	var theText;
			
	if (xhr.readyState == 4) {
		if (xhr.status == 200) {
			var allImages = xhr.responseXML.getElementsByTagName("content");

			for (var i=0; i<allImages.length; i++) {
				tempText.innerHTML = getPixVal(allImages[i]);

				theText = tempText.getElementsByTagName("p")[1].innerHTML;
				theText = theText.replace(/240/g,"75");
				theText = theText.replace(/180/g,"75");
				theText = theText.replace(/_m/g,"_s");
				document.getElementById("pictureBar").innerHTML += theText;
			}
		}
		else {
			alert("There was a problem with the request " + xhr.status);
		}
	}
	
	function getPixVal(inVal) {
		return (inVal.textContent) ? inVal.textContent : inVal.text;
	}
}


## 13.4 刷新服务器数据

In [ ]:
window.addEventListener("load",initAll,false);
var xhr = false;

function initAll() {
	if (window.XMLHttpRequest) {
		xhr = new XMLHttpRequest();
	}
	else {
		if (window.ActiveXObject) {
			try {
				xhr = new ActiveXObject("Microsoft.XMLHTTP");
			}
			catch (e) {
			}
		}
	}

	if (xhr) {
		getPix();
	}
	else {
		alert("Sorry, but I couldn't create an XMLHttpRequest");
	}
}

function getPix() {
	xhr.open("GET", "flickrfeed.xml", true);
	xhr.addEventListener("readystatechange",showPictures,false);
	xhr.send(null);

	setTimeout(getPix, 5 * 1000);
}

function showPictures() {
	var tempText = document.createElement("div");
			
	if (xhr.readyState == 4) {
		if (xhr.status == 200) {
			var allImages = xhr.responseXML.getElementsByTagName("content");
			var randomImg = Math.floor(Math.random() * allImages.length);

			tempText.innerHTML = getPixVal(allImages[randomImg]);
			var thisImg = tempText.getElementsByTagName("p")[1];
			document.getElementById("pictureBar").innerHTML = thisImg.innerHTML;
		}
		else {
			alert("There was a problem with the request " + xhr.status);
		}
	}
	
	function getPixVal(inVal) {
		return (inVal.textContent) ? inVal.textContent : inVal.text;
	}
}


## 13.5 从服务器获得数据

In [ ]:
<!DOCTYPE html>
<html>
<head>
	<title>Using JSON Data</title>
	<link rel="stylesheet" href="script02.css">
	<script src="script04.js"></script>
	<script src="http://api.flickr.com/services/feeds/photoset.gne?nsid=23922109@N00&set=72157600976524175&format=json"></script>
</head>
<body>
	<div id="pictureBar"> </div>
</body>
</html>

In [ ]:
window.addEventListener("load",initAll,false);
var imgDiv = "";

function initAll() {
	document.getElementById("pictureBar").innerHTML = imgDiv;
}

function jsonFlickrFeed(flickrData) {
	for (var i=0; i<flickrData.items.length; i++) {
		imgDiv += "<img src='";
		imgDiv += flickrData.items[i].media.m.replace(/_m/g,"_s");
		imgDiv += "' alt='" + flickrData.items[i].title + "'>";
	}
}


## 13.6 用 Ajax 预览链接


In [ ]:
window.addEventListener("load",initAll,false);
var xhr = false;
var xPos, yPos;

function initAll() {
	var allLinks = document.getElementsByTagName("a");

	for (var i=0; i< allLinks.length; i++) {
		allLinks[i].addEventListener("mouseover",getPreview,false);
	}
}

function getPreview(evt) {
	if (evt) {
		var url = evt.target;
	}
	else {
		evt = window.event;
		var url = evt.srcElement;
	}
	xPos = parseInt(evt.clientX);
	yPos = parseInt(evt.clientY);
	
	if (window.XMLHttpRequest) {
		xhr = new XMLHttpRequest();
	}
	else {
		if (window.ActiveXObject) {
			try {
				xhr = new ActiveXObject("Microsoft.XMLHTTP");
			}
			catch (e) {
			}
		}
	}

	if (xhr) {
		xhr.addEventListener("readystatechange",showContents,false);
		xhr.open("GET", url, true);
		xhr.send(null);
	}
	else {
		alert("Sorry, but I couldn't create an XMLHttpRequest");
	}
}

function showContents() {
	var prevWin = document.getElementById("previewWin");
	
	if (xhr.readyState == 4) {
		if (xhr.status == 200) {
			prevWin.innerHTML = xhr.responseText;
		}
		else {
			prevWin.innerHTML = "There was a problem with the request " + xhr.status;
		}
		prevWin.style.top = yPos+2 + "px";
		prevWin.style.left = xPos+2 + "px";
		prevWin.style.visibility = "visible";

		prevWin.addEventListener("mouseout", function() {prevWin.style.visibility = "hidden";}, false);
	}
}

## 13.7 自动补全表单字段

In [ ]:
window.addEventListener("load",initAll,false);
var xhr = false;
var statesArray = new Array();

function initAll() {
	document.getElementById("searchField").addEventListener("keyup",searchSuggest,false);

	if (window.XMLHttpRequest) {
		xhr = new XMLHttpRequest();
	}
	else {
		if (window.ActiveXObject) {
			try {
				xhr = new ActiveXObject("Microsoft.XMLHTTP");
			}
			catch (e) {
			}
		}
	}

	if (xhr) {
		xhr.addEventListener("readystatechange",setStatesArray,false);
		xhr.open("GET", "us-states.xml", true);
		xhr.send(null);
	}
	else {
		alert("Sorry, but I couldn't create an XMLHttpRequest");
	}
}

function setStatesArray() {
	if (xhr.readyState == 4) {
		if (xhr.status == 200) {
			if (xhr.responseXML) {
				var allStates = xhr.responseXML.getElementsByTagName("item");
				for (var i=0; i<allStates.length; i++) {
					statesArray[i] = allStates[i].getElementsByTagName("label")[0].firstChild;
				}
			}
		}
		else {
			alert("There was a problem with the request " + xhr.status);
		}
	}
}

function searchSuggest() {
	var str = document.getElementById("searchField").value;
	document.getElementById("searchField").className = "";
	if (str != "") {
		document.getElementById("popups").innerHTML = "";
	
		for (var i=0; i<statesArray.length; i++) {
			var thisState = statesArray[i].nodeValue;
	
			if (thisState.toLowerCase().indexOf(str.toLowerCase()) == 0) {
				var tempDiv = document.createElement("div");
				tempDiv.innerHTML = thisState;
				tempDiv.addEventListener("click",makeChoice,false);
				tempDiv.className = "suggestions";
				document.getElementById("popups").appendChild(tempDiv);
			}
		}
		var foundCt = document.getElementById("popups").childNodes.length;
		if (foundCt == 0) {
			document.getElementById("searchField").className = "error";
		}
		if (foundCt == 1) {
			document.getElementById("searchField").value = document.getElementById("popups").firstChild.innerHTML;
			document.getElementById("popups").innerHTML = "";
		}
	}
}

function makeChoice(evt) {
	if (evt) {	
		var thisDiv = evt.target;
	}
	else {
		var thisDiv = window.event.srcElement;
	}
	document.getElementById("searchField").value = thisDiv.innerHTML;
	document.getElementById("popups").innerHTML = "";
}
